## Report trend analysis for past *n* days

Import necessary packages

In [2]:
import os
import numpy as np
import pandas as pd
from scipy import stats
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression

In [15]:
df = pd.read_csv('../daily_prices/AAPL.csv')

# Decompose
result = sm.tsa.seasonal_decompose(df['Close'], model='additive', period=252, extrapolate_trend=25, two_sided=False)
trend = result.trend[-20:].reset_index(drop=True)

# normalize input data
mins, maxs = trend.min(), trend.max()
trend = (trend-mins)/(maxs-mins)
model = LinearRegression()
model.fit(np.array(trend.index.to_list()).reshape(-1, 1), trend.values.reshape(-1, 1))
model.coef_

array([[0.0551838]])

Analyze trends

In [30]:
# Read files from folder
folder = '../daily_prices'
files = os.listdir(folder)

# Create dataframe
trends = pd.DataFrame(columns=['Stock', 'Trend'])

for i, file in enumerate(files):
    # Read file
    df = pd.read_csv(os.path.join(folder, file))

    if not df.isna().any().any() and len(df.index) >= 504:
        # Decompose
        result = sm.tsa.seasonal_decompose(df['Close'], model='additive', period=252, extrapolate_trend=25, two_sided=False)
        trend = result.trend[-20:].reset_index(drop=True)

        # normalize input data
        # mins, maxs = trend.min(), trend.max()
        # trend = (trend-mins)/(maxs-mins)

        model = LinearRegression()
        model.fit(np.array(trend.index.to_list()).reshape(-1, 1), trend.values.reshape(-1, 1))
        slope = model.coef_[0][0]/trend.max()

        # Add to dataframe
        trends = pd.concat([trends, pd.DataFrame({'Stock': [file[:-4]], 'Trend': [slope]})], ignore_index=True)

trends.sort_values(by=['Trend'], ascending=False).reset_index(drop=True)

,Stock,Trend
0,NVDA,0.003784
1,META,0.003635
2,PHM,0.002470
3,FICO,0.002354
4,ADBE,0.002351
...,...,...
492,DG,-0.002678
493,ALB,-0.003004
494,ENPH,-0.004230
495,DISH,-0.004505


In [21]:
x = pd.DataFrame([[2,3,4]], columns=['A','B','C'])
y = pd.DataFrame([[5,6,7]], columns=['A','B','C'])
pd.concat([x, y])

,A,B,C
0,2,3,4
0,5,6,7
